In [10]:
import h5py
import matplotlib.pyplot as plt
import random
import pandas
import numpy as np
import os
from astropy import units as u
import SciServer.CasJobs as cj

In [2]:
import HMpTy
from astropy_healpix import HEALPix

In [3]:
folder='/home/idies/workspace/virgo_filedb/data05_01/apogee_fire/'
csvfolder=f'/home/idies/workspace/virgo/apogee_fire/CSV/'
os.makedirs(csvfolder,exist_ok=True)

# file='lsr-2-rslice-1.m12i-res7100-md-sliced-gcat-dr2-apogee-2mass-test.hdf5'
file='lsr-2-rslice-1.m12i-res7100-md-sliced-gcat-dr2-apogee-2mass-test-orig-dtype.hdf5'

path=f'{folder}{file}'
csvfile=f'{csvfolder}{file.replace(".hdf5",".wHTMetc.csv")}'
csvfile

'/home/idies/workspace/virgo/apogee_fire/CSV/lsr-2-rslice-1.m12i-res7100-md-sliced-gcat-dr2-apogee-2mass-test-orig-dtype.wHTMetc.csv'

In [4]:
hdf5_file=h5py.File(path,mode='r')

In [6]:
# depth 20 corresponds to HTM indexes in SDSS CAS databases
htm20 = HMpTy.HTM(depth=20)

In [7]:
# determine healpix nside required to get pixel area close to HTM pixel area
nside=int(pow(2,np.round(np.log2(np.sqrt(4*np.pi/12/(htm20.area*np.square(np.pi/180)))))))
print("nside=",nside," depth=",np.round(np.log2(nside)))
healp = HEALPix(nside=nside, order='nested')
print("HEALPIx resolution in arcsec:",healp.pixel_resolution.value*60)

nside= 1048576  depth= 20.0
HEALPIx resolution in arcsec: 0.20129803194242613


In [56]:

keys=hdf5_file.keys()
# reorder columns so that source_id is first
ndim=len(hdf5_file['source_id'])
cols=['source_id']+[c for c in keys if c not in ['source_id']]

In [65]:
%%time
header = True
batch=100000
i=0
while True:
    df = pandas.DataFrame()
    n=min(i+batch,ndim-1)
    for key in cols:
        df[key]=hdf5_file[key][i:n]
    df['htmid']=htm20.lookup_id(df['ra'],df['dec'])
    df['healpixId']=healp.lonlat_to_healpix(df['l'].values*u.deg,df['b'].values*u.deg)
    mode='a'
    if header:
        mode='w'
    df.to_csv(csvfile, header=header, mode=mode,index=False)
    i+=batch
    print(i)
    header = False
    if i>= ndim:
        break

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
CPU times: user 13min 38s, sys: 13.4 s, total: 13min 52s
Wall time: 14min 7s


# define DDL

In [59]:
typed_cols=zip(df.columns,[str(t) for t in df.dtypes])

In [50]:
def gettype(t):
    if t == "float32":
        return "REAL"
    elif t == "float64":
        return "FLOAT"
    elif t == "uint16":
        return 'SMALLINT'
    elif t == "int16":
        return 'SMALLINT'
    elif t == "uint64":
        return "BIGINT"
    elif t == "int64":
        return "BIGINT"
    return t

In [60]:
TABLE='TEST_M12i_v2'
columns=[f"[{c}] {gettype(t)}" for c,t in typed_cols]
newline='\n'
ddl=f"CREATE TABLE {TABLE} ({newline}{(newline+',  ').join(columns)})"
print(ddl)

CREATE TABLE TEST_M12i_v2 (
[source_id] BIGINT
,  [2MASS_magH] REAL
,  [2MASS_magH_error] REAL
,  [2MASS_magH_int] REAL
,  [2MASS_magH_true] REAL
,  [2MASS_magJ] REAL
,  [2MASS_magJ_error] REAL
,  [2MASS_magJ_int] REAL
,  [2MASS_magJ_true] REAL
,  [2MASS_magKs] REAL
,  [2MASS_magKs_error] REAL
,  [2MASS_magKs_int] REAL
,  [2MASS_magKs_true] REAL
,  [A0] REAL
,  [CFe-APOGEE] REAL
,  [CaFe-APOGEE] REAL
,  [FeH-APOGEE] REAL
,  [MgFe-APOGEE] REAL
,  [NFe-APOGEE] REAL
,  [OFe-APOGEE] REAL
,  [SFe-APOGEE] REAL
,  [SiFe-APOGEE] REAL
,  [a_g_bp_val] REAL
,  [a_g_rp_val] REAL
,  [a_g_val] REAL
,  [age] REAL
,  [alpha] REAL
,  [b] FLOAT
,  [b_true] FLOAT
,  [bp_g] REAL
,  [bp_g_int] REAL
,  [bp_g_true] REAL
,  [bp_rp] REAL
,  [bp_rp_int] REAL
,  [bp_rp_true] REAL
,  [calcium] REAL
,  [carbon] REAL
,  [dec] FLOAT
,  [dec_error] FLOAT
,  [dec_true] FLOAT
,  [dhel_true] FLOAT
,  [dmod_true] FLOAT
,  [e_bp_min_rp_val] REAL
,  [ebv] REAL
,  [feh] REAL
,  [g_rp] REAL
,  [g_rp_int] REAL
,  [g_rp_true] 

In [61]:
# query for counts of nulls for each column
columns='\n,      '.join([f"sum(case when [{key}] is null then 1 else 0 end) as [{key}]" for key in cols])
newline='\n'
sql=f"""
select {columns} 
  from {TABLE} """
print(sql)


select sum(case when [source_id] is null then 1 else 0 end) as [source_id]
,      sum(case when [2MASS_magH] is null then 1 else 0 end) as [2MASS_magH]
,      sum(case when [2MASS_magH_error] is null then 1 else 0 end) as [2MASS_magH_error]
,      sum(case when [2MASS_magH_int] is null then 1 else 0 end) as [2MASS_magH_int]
,      sum(case when [2MASS_magH_true] is null then 1 else 0 end) as [2MASS_magH_true]
,      sum(case when [2MASS_magJ] is null then 1 else 0 end) as [2MASS_magJ]
,      sum(case when [2MASS_magJ_error] is null then 1 else 0 end) as [2MASS_magJ_error]
,      sum(case when [2MASS_magJ_int] is null then 1 else 0 end) as [2MASS_magJ_int]
,      sum(case when [2MASS_magJ_true] is null then 1 else 0 end) as [2MASS_magJ_true]
,      sum(case when [2MASS_magKs] is null then 1 else 0 end) as [2MASS_magKs]
,      sum(case when [2MASS_magKs_error] is null then 1 else 0 end) as [2MASS_magKs_error]
,      sum(case when [2MASS_magKs_int] is null then 1 else 0 end) as [2MASS_ma

In [62]:
df=cj.executeQuery(sql,"ApogeeFire")

In [63]:
row=df.iloc[0]
for k in df.columns:
    if row[k] != 0:
        print(k,'\t\t',row[k])

CFe-APOGEE 		 5109
CaFe-APOGEE 		 5109
FeH-APOGEE 		 5109
MgFe-APOGEE 		 5109
NFe-APOGEE 		 5109
OFe-APOGEE 		 5109
SFe-APOGEE 		 5109
SiFe-APOGEE 		 5109
radial_velocity 		 9976
radial_velocity_error 		 9976
